In [55]:
import pandas as pd
import os
import copy
import numpy as np
import re
from datetime import datetime

# Pull Plan Algorithm for Port HU #

In [58]:
ref_port = 'HU'

In [39]:
#Vehicle List by VIN and Model Type
col_list = ["VIN", "VEHICLE_TYPE", "MODEL_TYPE", "MODEL_CODE", "DEPT_PORT"]
iter_csv = pd.read_csv(r'\\hke.local\Glovis\Public\Power BI\MASTER LISTS\Vehicle List\vehicle_list_rolling_year.csv', iterator=True, chunksize=4000, usecols=col_list)
vl_pull = pd.concat(chunk.loc[(chunk['DEPT_PORT'] == ref_port)] for chunk in iter_csv)
vl_mdl_cd = vl_pull

In [40]:
#HU ANT list
ant = pd.read_excel(r'\\hke.local\Glovis\Dept\PI\HQ Projects\PP Optimization 2022\iVELES Available Manual Pull 012722.xlsx')

In [41]:
#HU ANT list
lts_mapping = pd.read_excel(r'\\hke.local\Glovis\Dept\PI\HQ Projects\PP Optimization 2022\ITMS Code LTS Mapping Contract.xlsx')

In [42]:
pd.set_option('display.max_columns', None)

In [43]:
lts_mapping

,Port,Customer,No,Rate Start Date,Rate End Date,WO Code,Description,Cost Flag,Vehicle Type,Model Type,Accry. Lbr. Time,PDI Time,Warranty Time,Body Shop Time,Fixed Rate,Mark Up Rate,Rate,Skilled YN,Use YN
0,HU,HMA,1,01/01/2022,12/31/2022,77,Load Capacity Labels - Installs only,No,R5 - Palisade,Palisade,0.025,0.0,0,0,0.0,0,0.72225,N,Yes
1,HU,HMA,2,01/01/2022,12/31/2022,75,Tow Hook Removal (2),No,R5 - Palisade,Palisade,0.120,0.0,0,0,0.0,0,3.98640,N,Yes
2,HU,HMA,3,01/01/2022,12/31/2022,74,Tow Hook Removal (1),No,R5 - Palisade,Palisade,0.060,0.0,0,0,0.0,0,1.99320,N,Yes
3,HU,HMA,4,01/01/2022,12/31/2022,69,XM Radio Brochure,No,R5 - Palisade,Palisade,0.005,0.0,0,0,0.0,0,0.14445,N,Yes
4,HU,HMA,5,01/01/2022,12/31/2022,68,XM Check at Final Inspection,No,R5 - Palisade,Palisade,0.050,0.0,0,0,0.0,0,1.44450,N,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3475,HU,KMA,2352,01/01/2021,12/31/2021,12,Collect Key Code,No,E3 - Stinger,Stinger,0.050,0.0,0,0,0.0,0,2.11700,N,Yes
3476,HU,KMA,2353,01/01/2021,12/31/2021,12,Collect Key Code,No,E2 - Stinger,Stinger,0.050,0.0,0,0,0.0,0,2.11700,N,Yes
3477,HU,KMA,2354,01/01/2021,12/31/2021,12,Collect Key Code,No,E1 - Stinger,Stinger,0.050,0.0,0,0,0.0,0,2.11700,N,Yes
3478,HU,KMA,2355,01/01/2021,12/31/2021,12,Collect Key Code,No,CM - Niro PHEV,Niro PHEV,0.050,0.0,0,0,0.0,0,2.11700,N,Yes


In [44]:
lts_mapping['Vehicle Type'] = [cd[0:cd.find(' ')] for cd in lts_mapping['Vehicle Type']]

In [45]:
ant_vtype = pd.merge(ant, vl_mdl_cd[['VEHICLE_TYPE']], left_on = ant['VIN'], right_on = vl_mdl_cd['VIN'], how = 'left')

In [63]:
ant_vtype = ant_vtype.drop(columns = ['No', 'key_0'])

In [64]:
ant_vtype

,No,Port,VIN,Status,Customer,VPS,Mani Aloc Date,Mani Aloc No.,Model Type,Model Year,Model,Acce Grup Cd,Carrier,Dealer,Dest,Req Port Rels Date,Ship To Dealer,Ship.Dealer Type,Ship to City,Ship to State,Region,Ext. Color,Int. Color,Production Date,Received,Allocation,Allocation Date,Allocated,Sold Status,Instruction,Drop Ship,VPC In Date,VPC Out Date,Release Date,Shipment Date,Shipment No.,Load No.,Load Date,Rail Load Position,Trans. Type,De-commit Date,PUP No.,PUP Date,PUP Open Flag,Activity Code,PIO Code,Install Type,HOLD,HOLD Code,HOLD Comment,R/O No.,R/O Date,Start Due Date,End Due Date,Work Order Print Date,Received Aged,Allocated Days Aged,comment,Location,Location Update,Location Update Date,Vessel,Vessel ETA,In Process Date,Arrival Date,Rail Car No.,XM ID,Inspector ID,Throw In ID,VEHICLE_TYPE,Light PIO Hrs,Heavy PIO Hrs
646,647,HU,KNDRG4LG6N5081941,[N]Normal Port Stock,KMA,[S2000]Stock,11/03/2021,2021120200262,[SRH]SORENTO HEV (MQ4),2022,[U4232]MY22 SORENTO HEV (MQ4),10,"[EXTRM]Extreme Transportation, Inc.",[CA107]North County Kia,NaN,12/19/2021,[CA107]North County Kia,Dealer Car,ESCONDIDO,CA,[WE][KMA/HMA] Western,[SWP]Snow White Pearl,[WK]Black,11/22/2021,12/17/2021,P202111001,11/03/2021,Y,[U]Unsold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,105 MQBAX1 MQEH02 MQHCF1 MQIL02 MQMUG2 MQRSE1,"08,105,18,19,25,35,68,BAX,CF,EH,IL,MUG,RSE",Heavy PIO,No,NaN,No cmnt hold,NaN,NaN,12/19/2021,12/20/2021,01/26/2022 14:46:15,42,41,NaN,02-01-001-001,kcurran,01/25/2022 07:44:03,MORNING,12/16/2021,12/20/2021,12/16/2021,NaN,NaN,NaN,NaN,NaN,0.96,0
647,648,HU,KNDRG4LG6N5081972,[N]Normal Port Stock,KMA,[S2000]Stock,11/03/2021,2021120200262,[SRH]SORENTO HEV (MQ4),2022,[U4232]MY22 SORENTO HEV (MQ4),10,[MPG]United Road,[NV009]Findlay Kia,NaN,12/19/2021,[NV009]Findlay Kia,Dealer Car,LAS VEGAS,NV,[WE][KMA/HMA] Western,[SWP]Snow White Pearl,[GYT]Gray,11/22/2021,12/17/2021,P202112001,11/03/2021,Y,[U]Unsold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,MQCTS1 MQEH02 MQHCF1,"08,18,19,25,35,68,CF,CTS,EH",Heavy PIO,No,NaN,No cmnt hold,NaN,NaN,12/19/2021,12/20/2021,01/26/2022 14:46:15,42,41,NaN,02-01-001-001,kcurran,01/25/2022 07:44:06,MORNING,12/16/2021,12/20/2021,12/16/2021,NaN,NaN,NaN,NaN,NaN,0.46,0
649,650,HU,KNDJ23AU1N7177138,[N]Normal Port Stock,KMA,[S2000]Stock,01/03/2022,2022010300036,[SO1]SOUL (SK3),2022,[B2522]MY22 SOUL (SK3),15,"[EXCEL]Deluxe Auto Carriers, Inc.",[CA305B]KIA OF VACAVILLE,NaN,01/06/2022,[DS1413]AVIS RENT A CAR,Fleet Car,SOUTH SAN FRANCISCO,CA,[WF][KMA] Fleet,[SWP]Snow White Pearl,[WK]Black,12/01/2021,12/17/2021,0017894745,01/03/2022,Y,[U]Unsold,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,105 SKCF02,"08,105,18,19,25,30,35,CF",Light PIO,No,NaN,No cmnt hold,NaN,NaN,01/06/2022,01/15/2022,01/19/2022 15:58:12,42,24,NaN,02-01-001-001,kcurran,01/25/2022 07:44:06,MORNING,12/16/2021,01/04/2022,12/16/2021,NaN,NaN,NaN,NaN,NaN,0.00,0
447,448,HU,KNDJ23AU4N7176596,[N]Normal Port Stock,KMA,[S2000]Stock,01/03/2022,2022010300036,[SO1]SOUL (SK3),2022,[B2522]MY22 SOUL (SK3),15,[MPG]United Road,[CA305B]KIA OF VACAVILLE,NaN,01/06/2022,[DS1382]AVIS RENT A CAR,Fleet Car,PHOENIX,AZ,[WF][KMA] Fleet,[SWP]Snow White Pearl,[WK]Black,11/29/2021,12/17/2021,0017894626,01/03/2022,Y,[U]Unsold,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,SKCF02,"08,18,19,25,30,35,CF",Throw-in PIO,No,NaN,No cmnt hold,NaN,NaN,01/06/2022,01/15/2022,01/19/2022 15:58:12,42,24,NaN,02-01-001-001,kcurran,01/25/2022 07:44:06,MORNING,12/16/2021,01/04/2022,12/16/2021,NaN,NaN,NaN,NaN,NaN,0.00,0
446,447,HU,KNDJ23AU2N7176161,[N]Normal Port Stock,KMA,[S2000]Stock,01/03/2022,2022010300036,[SO1]SOUL (SK3),2022,[B2522]MY22 SOUL (SK3),15,"[EXTRM]Extreme Transportation, Inc.",[CA305B]KIA OF VACAVILLE,NaN,01/06/2022,[DS1872]AVIS RENT A CAR,Fleet Car,SAN DIEGO,CA,[WF][KMA] Fleet,[SWP]Snow White Pearl,[WK]Black,11/26/2021,12/17/2021,0017894844,01/03/2022,Y,[U]Unsold,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,105 SKCF02,"08,

In [47]:
def average(lst):
    if len(lst) > 0:
        avg = sum(lst) / len(lst)
    else:
        avg = 0
    return avg

In [48]:
#LTS Mapping - direct lookup for one-to-one match b/w vehicle type and PIO code, average calculation otherwise
light_pio_hrs = []
heavy_pio_hrs = []

for i in range(0, len(ant_vtype)):
    
    #If there is no PIO code, put empty list into PIO_code_iter. Else break up comma-delimited entry to list
    if pd.isnull(ant_vtype['PIO Code'][i]):
        PIO_code_iter = []
    else:
        PIO_code_iter = ant_vtype['PIO Code'][i].split(',')
    
    #Total light and heavy PIO hours
    light_pio_entry = 0
    heavy_pio_entry = 0
    
    #For codes in PIO_code_iter, calculate LTS for non-numeric codes only (PIO)
    for cd in PIO_code_iter:
        if not cd.isnumeric():
            vtype = ant_vtype['VEHICLE_TYPE'][i]
            cust = ant_vtype['Customer'][i]
            
            #Filter down LTS table by vehicle type, WO code, and customer
            lts_restr = list(lts_mapping.loc[(lts_mapping['Vehicle Type'] == vtype) & (lts_mapping['WO Code'] == cd) & (lts_mapping['Customer'] == cust)]['Accry. Lbr. Time'])
            
            #If clean match, put LTS between 0.05 and 0.5 into light, and above 0.5 into heavy (<= 0.05 are throw-ins)
            if len(lts_restr) == 1:
                if(lts_restr[0] < 0.5 and lts_restr[0] > 0.05):
                    light_pio_entry += lts_restr[0]
                elif (lts_restr[0] >= 0.5):
                    heavy_pio_entry += lts_restr[0]
            
            #Else repeat but apply average for WO code and customer
            else:
                lts_restr = list(lts_mapping.loc[(lts_mapping['WO Code'] == cd) & (lts_mapping['Customer'] == cust)]['Accry. Lbr. Time'])
                
                if(average(lts_restr) < 0.5 and average(lts_restr) > 0.05):
                    light_pio_entry += average(lts_restr)
                elif (average(hu_lts_restr) >= 0.5):
                    heavy_pio_entry += average(lts_restr)
            
    light_pio_hrs.append(round(light_pio_entry, 2))
    heavy_pio_hrs.append(round(heavy_pio_entry, 2))        

In [50]:
ant_vtype['Light PIO Hrs'] = light_pio_hrs
ant_vtype['Heavy PIO Hrs'] = heavy_pio_hrs

In [51]:
ant_vtype

,key_0,No,Port,VIN,Status,Customer,VPS,Mani Aloc Date,Mani Aloc No.,Model Type,Model Year,Model,Acce Grup Cd,Carrier,Dealer,Dest,Req Port Rels Date,Ship To Dealer,Ship.Dealer Type,Ship to City,Ship to State,Region,Ext. Color,Int. Color,Production Date,Received,Allocation,Allocation Date,Allocated,Sold Status,Instruction,Drop Ship,VPC In Date,VPC Out Date,Release Date,Shipment Date,Shipment No.,Load No.,Load Date,Rail Load Position,Trans. Type,De-commit Date,PUP No.,PUP Date,PUP Open Flag,Activity Code,PIO Code,Install Type,HOLD,HOLD Code,HOLD Comment,R/O No.,R/O Date,Start Due Date,End Due Date,Work Order Print Date,Received Aged,Allocated Days Aged,comment,Location,Location Update,Location Update Date,Vessel,Vessel ETA,In Process Date,Arrival Date,Rail Car No.,XM ID,Inspector ID,Throw In ID,VEHICLE_TYPE,Light PIO Hrs,Heavy PIO Hrs
0,KM8KM4AE9NU052589,1,HU,KM8KM4AE9NU052589,[N]Normal Port Stock,HMA,[S2000]Stock,12/16/2021,2021121600002,[IQ5]IONIQ 5 (NE1),2022,[50412REZ]MY22 IONIQ 5 (NE1),1,[MPG]United Road,[AZ043]EARNHARDT HYUNDAI NORTH SCOTTS,NaN,NaN,[AZ043]EARNHARDT HYUNDAI NORTH SCOTTS,Dealer Car,SCOTTSDALE,AZ,[MS],[SAW]Lunar White,[YGN]Gray,11/09/2021,12/09/2021,3667386,12/16/2021,Y,[U]Unsold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,MUFK02 NECFF1 NECN01 NECT01,"CF,CN,CT,FK",Throw-in PIO,No,NaN,No cmnt hold,NaN,NaN,NaN,NaN,01/21/2022 13:54:41,50,42,NaN,15-06-050-003,aflores,01/05/2022 09:44:48,MSC CRIS,12/05/2021,12/20/2021,12/08/2021,NaN,NaN,NaN,NaN,NaN,0.23,0
1,KM8S6DA25NU033540,2,HU,KM8S6DA25NU033540,[N]Normal Port Stock,HMA,[S2000]Stock,12/22/2021,2022010500011,[SFP]SANTA FE PHEV (TMa),2022,[644D2APT]MY22 SANTA FE PHEV (TMa),1,"[EXTRM]Extreme Transportation, Inc.",[CA320]DUBLIN HYUNDAI,NaN,NaN,[CA320]DUBLIN HYUNDAI,Dealer Car,DUBLIN,CA,[WE][KMA/HMA] Western,[WW8]Quartz White,[NNB]Black,12/08/2021,01/11/2022,3680133,12/22/2021,Y,[S]Sold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,97 MUCFK5 TMCCN1 TMCCT2 TMCV01 TMHCF1,"C1,CF,CV",Light PIO,No,NaN,No cmnt hold,NaN,NaN,NaN,NaN,NaN,17,16,NaN,1C-WA-013-001,jlamb,01/11/2022 09:41:43,LAKE GEN,01/05/2022,01/12/2022,01/08/2022,NaN,NaN,NaN,NaN,NaN,0.30,0
2,KMHT36AH1NU013136,3,HU,KMHT36AH1NU013136,[N]Normal Port Stock,HMA,[S2000]Stock,12/22/2021,2021122200307,[VL1]VELOSTER (JS),2022,[FN352FT5]MY22 VELOSTER N (JS),1,"[EXTRM]Extreme Transportation, Inc.",[CA323]NORM REEVES HYU SUPERSTORE CER,NaN,NaN,[CA323]NORM REEVES HYU SUPERSTORE CER,Dealer Car,CERRITOS,CA,[WE][KMA/HMA] Western,[MZH]Ultra Black,[NNB]Black,12/13/2021,01/11/2022,3679567,12/22/2021,Y,[U]Unsold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,97 JSCT01 JSPC01,"CT,PC",Light PIO,No,NaN,No cmnt hold,NaN,NaN,NaN,NaN,01/21/2022 15:25:36,17,16,NaN,04-06-081-001,jlamb,01/11/2022 09:50:41,LAKE GEN,01/05/2022,01/12/2022,01/08/2022,NaN,NaN,NaN,NaN,NaN,0.10,0
3,KNDJ23AU4N7821344,4,HU,KNDJ23AU4N7821344,[N]Normal Port Stock,KMA,[S2000]Stock,11/19/2021,2021122300745,[SO1]SOUL (SK3),2022,[B2522]MY22 SOUL (SK3),10,[MPG]United Road,[NV012]Towbin Kia,NaN,01/18/2022,[NV012]Towbin Kia,Dealer Car,HENDERSON,NV,[WE][KMA/HMA] Western,[KDT]Gravity Grey,[WK]Black,12/11/2021,01/20/2022,P202111001,11/19/2021,Y,[U]Unsold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,SKCF02 SKCN02,"08,18,19,25,35,CF,CN",Light PIO,No,NaN,No cmnt hold,NaN,NaN,01/18/2022,01/20/2022,NaN,8,7,NaN,07-02-055-001,wfredrick,01/20/2022 13:19:48,DIGNITY,01/15/2022,01/21/2022,01/18/2022,NaN,NaN,NaN,NaN,NaN,0.11,0
4,KNDJ23AU6N7822270,5,HU,KNDJ23AU6N7822270,[N]Normal Port Stock,KMA,[S2000]Stock,11/23/2021,2021122300745,[SO1]SOUL (SK3),2022,[B2522]MY22 SOUL (SK3),10,[MPG]United Road,[AZ049]PEORIA KIA,NaN,01/18/2022,[AZ049]PEORIA KIA,Dealer Car,PEORIA,AZ,[WE][KMA/HMA] Western,[M4A]Mars Orange,[GDM]Black/Gray,12/20/2021,01/20/2022,P202112001,11/23/2021,Y,[U]Unsold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,SKCF02,"08,18,19,25,35,CF",Throw-in PIO,No,NaN,No cmnt hold,NaN,NaN,01/18/2022,01/19/2022,NaN,8,7,NaN,

In [59]:
"""
Allocation Hours Paycom Data
"""
# workstation lookup dictionary
def read_workstation():
    os.chdir('G:\PI\HQ Projects\PP Optimization 2022')
    workstation = pd.read_excel('WorkstationHR.xlsx')
    # workstation dictionary created
    ws = workstation.groupby('Port')[['Workstation','Category']].apply(lambda x: x.set_index('Workstation').to_dict(orient='index')).to_dict()
    return ws

# last week paycom data
def read_lastweek():
    # holiday table
    os.chdir('G:\PI\HQ Projects\PP Optimization 2022')
    holiday = pd.read_excel('HolidayTable.xlsx')
    holiday['Date'] = pd.to_datetime(holiday['Date'], format='%m/%d/%Y')
    # allocation excel files
    os.chdir('P:\Cost Accounting\Alloc Dept Time detail\Allocation Hours')
    files = [f for f in os.listdir('P:\Cost Accounting\Alloc Dept Time detail\Allocation Hours') if f.startswith('A')]
    files.sort()
    
    df = pd.DataFrame()
    days = 0
    for f in files[-7:]:
        parsed_dates = re.findall("\d+", f)
        dates = (datetime.strptime(parsed_dates[0], "%Y%m%d")).date()
        dates_week = dates.weekday()
        if (dates not in list(holiday['Date'].dt.date)) and (dates_week != 5 and dates_week != 6):
            temp_df = pd.read_excel(f)
            df = df.append(temp_df,ignore_index=True)
            days+=1
    
    # map payroll code with port name, add workstation category using dictionary
    conditions = [ df['Allocation Payroll Profile Code'] =='0U376', df['Allocation Payroll Profile Code'] =='0U377', 
               df['Allocation Payroll Profile Code'] =='0U378',df['Allocation Payroll Profile Code'] =='0U379',
               df['Allocation Payroll Profile Code'] =='0NA37']
    choices = [ "MA", "HU", "PH", "WP", "SH"]
    
    df['Port'] = np.select(conditions, choices, default=np.nan)
    df = df[df['Port']!='nan']
    df = df[['Allocation Payroll Profile Code','EE Code', 'Earn Code','Earn Hours','DATE','Allocation Department Detail Desc','Allocation Company Desc','Port']]

    # map ws dictionary to paycom dataframe
    w = read_workstation()
    df['Cost_category'] = df.apply(lambda x: w[x['Port']][x['Allocation Department Detail Desc']]['Category'], axis=1)
    
    #df_clean = df[ (df['Cost_category'] =='PDI') | (df['Cost_category'] =='PIO')]
    #display(df_clean.groupby(['Port','Cost_category','Earn Code']).sum())

    # separate pdi and pio working hours per day
    # SD1 is regular hours code for agency employees
    df_pdi = df[(df['Cost_category'] =='PDI') & (df['Earn Code'] == 'R') | (df['Cost_category'] =='PDI') & (df['Earn Code'] == 'SD1')]
    df_pio = df[(df['Cost_category'] =='PIO') & (df['Earn Code'] == 'R') | (df['Cost_category'] =='PIO') & (df['Earn Code'] == 'SD1')]
    
    # divided by days to get daily hours
    pdi_hours = round(df_pdi.groupby(['Port']).sum()/days,2)
    pdi_hours = pdi_hours.rename(columns={"Earn Hours": "PDIHoursperDay"}).reset_index()
    pio_hours = round(df_pio.groupby(['Port']).sum()/days,2)
    pio_hours = pio_hours.rename(columns={"Earn Hours": "PIOHoursperDay"}).reset_index()

    # total working hours added to dataframe
    hours = pdi_hours.merge(pio_hours, on="Port", how="left")
    hours['TotalHours'] = hours['PIOHoursperDay'] + hours['PDIHoursperDay']

    return hours

ws = read_workstation()
hours = read_lastweek()
hours = hours.loc[hours['Port'] == ref_port]
hours_list = hours.values.tolist()

display(hours)
hours_list

,Port,PDIHoursperDay,PIOHoursperDay,TotalHours
0,HU,117.91,42.53,160.44


[['HU', 117.91, 42.53, 160.44]]

In [66]:
ant_vtype = ant_vtype.sort_values(by = 'Location')
ant_vtype

,Port,VIN,Status,Customer,VPS,Mani Aloc Date,Mani Aloc No.,Model Type,Model Year,Model,Acce Grup Cd,Carrier,Dealer,Dest,Req Port Rels Date,Ship To Dealer,Ship.Dealer Type,Ship to City,Ship to State,Region,Ext. Color,Int. Color,Production Date,Received,Allocation,Allocation Date,Allocated,Sold Status,Instruction,Drop Ship,VPC In Date,VPC Out Date,Release Date,Shipment Date,Shipment No.,Load No.,Load Date,Rail Load Position,Trans. Type,De-commit Date,PUP No.,PUP Date,PUP Open Flag,Activity Code,PIO Code,Install Type,HOLD,HOLD Code,HOLD Comment,R/O No.,R/O Date,Start Due Date,End Due Date,Work Order Print Date,Received Aged,Allocated Days Aged,comment,Location,Location Update,Location Update Date,Vessel,Vessel ETA,In Process Date,Arrival Date,Rail Car No.,XM ID,Inspector ID,Throw In ID,VEHICLE_TYPE,Light PIO Hrs,Heavy PIO Hrs
646,HU,KNDRG4LG6N5081941,[N]Normal Port Stock,KMA,[S2000]Stock,11/03/2021,2021120200262,[SRH]SORENTO HEV (MQ4),2022,[U4232]MY22 SORENTO HEV (MQ4),10,"[EXTRM]Extreme Transportation, Inc.",[CA107]North County Kia,NaN,12/19/2021,[CA107]North County Kia,Dealer Car,ESCONDIDO,CA,[WE][KMA/HMA] Western,[SWP]Snow White Pearl,[WK]Black,11/22/2021,12/17/2021,P202111001,11/03/2021,Y,[U]Unsold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,105 MQBAX1 MQEH02 MQHCF1 MQIL02 MQMUG2 MQRSE1,"08,105,18,19,25,35,68,BAX,CF,EH,IL,MUG,RSE",Heavy PIO,No,NaN,No cmnt hold,NaN,NaN,12/19/2021,12/20/2021,01/26/2022 14:46:15,42,41,NaN,02-01-001-001,kcurran,01/25/2022 07:44:03,MORNING,12/16/2021,12/20/2021,12/16/2021,NaN,NaN,NaN,NaN,NaN,0.96,0
647,HU,KNDRG4LG6N5081972,[N]Normal Port Stock,KMA,[S2000]Stock,11/03/2021,2021120200262,[SRH]SORENTO HEV (MQ4),2022,[U4232]MY22 SORENTO HEV (MQ4),10,[MPG]United Road,[NV009]Findlay Kia,NaN,12/19/2021,[NV009]Findlay Kia,Dealer Car,LAS VEGAS,NV,[WE][KMA/HMA] Western,[SWP]Snow White Pearl,[GYT]Gray,11/22/2021,12/17/2021,P202112001,11/03/2021,Y,[U]Unsold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,MQCTS1 MQEH02 MQHCF1,"08,18,19,25,35,68,CF,CTS,EH",Heavy PIO,No,NaN,No cmnt hold,NaN,NaN,12/19/2021,12/20/2021,01/26/2022 14:46:15,42,41,NaN,02-01-001-001,kcurran,01/25/2022 07:44:06,MORNING,12/16/2021,12/20/2021,12/16/2021,NaN,NaN,NaN,NaN,NaN,0.46,0
649,HU,KNDJ23AU1N7177138,[N]Normal Port Stock,KMA,[S2000]Stock,01/03/2022,2022010300036,[SO1]SOUL (SK3),2022,[B2522]MY22 SOUL (SK3),15,"[EXCEL]Deluxe Auto Carriers, Inc.",[CA305B]KIA OF VACAVILLE,NaN,01/06/2022,[DS1413]AVIS RENT A CAR,Fleet Car,SOUTH SAN FRANCISCO,CA,[WF][KMA] Fleet,[SWP]Snow White Pearl,[WK]Black,12/01/2021,12/17/2021,0017894745,01/03/2022,Y,[U]Unsold,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,105 SKCF02,"08,105,18,19,25,30,35,CF",Light PIO,No,NaN,No cmnt hold,NaN,NaN,01/06/2022,01/15/2022,01/19/2022 15:58:12,42,24,NaN,02-01-001-001,kcurran,01/25/2022 07:44:06,MORNING,12/16/2021,01/04/2022,12/16/2021,NaN,NaN,NaN,NaN,NaN,0.00,0
447,HU,KNDJ23AU4N7176596,[N]Normal Port Stock,KMA,[S2000]Stock,01/03/2022,2022010300036,[SO1]SOUL (SK3),2022,[B2522]MY22 SOUL (SK3),15,[MPG]United Road,[CA305B]KIA OF VACAVILLE,NaN,01/06/2022,[DS1382]AVIS RENT A CAR,Fleet Car,PHOENIX,AZ,[WF][KMA] Fleet,[SWP]Snow White Pearl,[WK]Black,11/29/2021,12/17/2021,0017894626,01/03/2022,Y,[U]Unsold,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,SKCF02,"08,18,19,25,30,35,CF",Throw-in PIO,No,NaN,No cmnt hold,NaN,NaN,01/06/2022,01/15/2022,01/19/2022 15:58:12,42,24,NaN,02-01-001-001,kcurran,01/25/2022 07:44:06,MORNING,12/16/2021,01/04/2022,12/16/2021,NaN,NaN,NaN,NaN,NaN,0.00,0
446,HU,KNDJ23AU2N7176161,[N]Normal Port Stock,KMA,[S2000]Stock,01/03/2022,2022010300036,[SO1]SOUL (SK3),2022,[B2522]MY22 SOUL (SK3),15,"[EXTRM]Extreme Transportation, Inc.",[CA305B]KIA OF VACAVILLE,NaN,01/06/2022,[DS1872]AVIS RENT A CAR,Fleet Car,SAN DIEGO,CA,[WF][KMA] Fleet,[SWP]Snow White Pearl,[WK]Black,11/26/2021,12/17/2021,0017894844,01/03/2022,Y,[U]Unsold,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,105 SKCF02,"08,105,18,19,25,30,35,CF",

In [67]:
ant_vtype['Allocation Date'] = pd.to_datetime(ant_vtype['Allocation Date'], format='%m/%d/%Y')
ant_vtype['Received'] = pd.to_datetime(ant_vtype['Received'], format='%m/%d/%Y')

In [71]:
ant_vtype['Start Date'] = ant_vtype[["Allocation Date", "Received"]].max(axis=1)

In [82]:
ant_vtype['ANT Aging'] = [delta.days for delta in (datetime.today() - ant_vtype['Start Date'])]

In [83]:
ant_vtype

,Port,VIN,Status,Customer,VPS,Mani Aloc Date,Mani Aloc No.,Model Type,Model Year,Model,Acce Grup Cd,Carrier,Dealer,Dest,Req Port Rels Date,Ship To Dealer,Ship.Dealer Type,Ship to City,Ship to State,Region,Ext. Color,Int. Color,Production Date,Received,Allocation,Allocation Date,Allocated,Sold Status,Instruction,Drop Ship,VPC In Date,VPC Out Date,Release Date,Shipment Date,Shipment No.,Load No.,Load Date,Rail Load Position,Trans. Type,De-commit Date,PUP No.,PUP Date,PUP Open Flag,Activity Code,PIO Code,Install Type,HOLD,HOLD Code,HOLD Comment,R/O No.,R/O Date,Start Due Date,End Due Date,Work Order Print Date,Received Aged,Allocated Days Aged,comment,Location,Location Update,Location Update Date,Vessel,Vessel ETA,In Process Date,Arrival Date,Rail Car No.,XM ID,Inspector ID,Throw In ID,VEHICLE_TYPE,Light PIO Hrs,Heavy PIO Hrs,Start Date,ANT Aging
646,HU,KNDRG4LG6N5081941,[N]Normal Port Stock,KMA,[S2000]Stock,11/03/2021,2021120200262,[SRH]SORENTO HEV (MQ4),2022,[U4232]MY22 SORENTO HEV (MQ4),10,"[EXTRM]Extreme Transportation, Inc.",[CA107]North County Kia,NaN,12/19/2021,[CA107]North County Kia,Dealer Car,ESCONDIDO,CA,[WE][KMA/HMA] Western,[SWP]Snow White Pearl,[WK]Black,11/22/2021,2021-12-17,P202111001,2021-11-03,Y,[U]Unsold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,105 MQBAX1 MQEH02 MQHCF1 MQIL02 MQMUG2 MQRSE1,"08,105,18,19,25,35,68,BAX,CF,EH,IL,MUG,RSE",Heavy PIO,No,NaN,No cmnt hold,NaN,NaN,12/19/2021,12/20/2021,01/26/2022 14:46:15,42,41,NaN,02-01-001-001,kcurran,01/25/2022 07:44:03,MORNING,12/16/2021,12/20/2021,12/16/2021,NaN,NaN,NaN,NaN,NaN,0.96,0,2021-12-17,52
647,HU,KNDRG4LG6N5081972,[N]Normal Port Stock,KMA,[S2000]Stock,11/03/2021,2021120200262,[SRH]SORENTO HEV (MQ4),2022,[U4232]MY22 SORENTO HEV (MQ4),10,[MPG]United Road,[NV009]Findlay Kia,NaN,12/19/2021,[NV009]Findlay Kia,Dealer Car,LAS VEGAS,NV,[WE][KMA/HMA] Western,[SWP]Snow White Pearl,[GYT]Gray,11/22/2021,2021-12-17,P202112001,2021-11-03,Y,[U]Unsold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,MQCTS1 MQEH02 MQHCF1,"08,18,19,25,35,68,CF,CTS,EH",Heavy PIO,No,NaN,No cmnt hold,NaN,NaN,12/19/2021,12/20/2021,01/26/2022 14:46:15,42,41,NaN,02-01-001-001,kcurran,01/25/2022 07:44:06,MORNING,12/16/2021,12/20/2021,12/16/2021,NaN,NaN,NaN,NaN,NaN,0.46,0,2021-12-17,52
649,HU,KNDJ23AU1N7177138,[N]Normal Port Stock,KMA,[S2000]Stock,01/03/2022,2022010300036,[SO1]SOUL (SK3),2022,[B2522]MY22 SOUL (SK3),15,"[EXCEL]Deluxe Auto Carriers, Inc.",[CA305B]KIA OF VACAVILLE,NaN,01/06/2022,[DS1413]AVIS RENT A CAR,Fleet Car,SOUTH SAN FRANCISCO,CA,[WF][KMA] Fleet,[SWP]Snow White Pearl,[WK]Black,12/01/2021,2021-12-17,0017894745,2022-01-03,Y,[U]Unsold,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,105 SKCF02,"08,105,18,19,25,30,35,CF",Light PIO,No,NaN,No cmnt hold,NaN,NaN,01/06/2022,01/15/2022,01/19/2022 15:58:12,42,24,NaN,02-01-001-001,kcurran,01/25/2022 07:44:06,MORNING,12/16/2021,01/04/2022,12/16/2021,NaN,NaN,NaN,NaN,NaN,0.00,0,2022-01-03,35
447,HU,KNDJ23AU4N7176596,[N]Normal Port Stock,KMA,[S2000]Stock,01/03/2022,2022010300036,[SO1]SOUL (SK3),2022,[B2522]MY22 SOUL (SK3),15,[MPG]United Road,[CA305B]KIA OF VACAVILLE,NaN,01/06/2022,[DS1382]AVIS RENT A CAR,Fleet Car,PHOENIX,AZ,[WF][KMA] Fleet,[SWP]Snow White Pearl,[WK]Black,11/29/2021,2021-12-17,0017894626,2022-01-03,Y,[U]Unsold,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,SKCF02,"08,18,19,25,30,35,CF",Throw-in PIO,No,NaN,No cmnt hold,NaN,NaN,01/06/2022,01/15/2022,01/19/2022 15:58:12,42,24,NaN,02-01-001-001,kcurran,01/25/2022 07:44:06,MORNING,12/16/2021,01/04/2022,12/16/2021,NaN,NaN,NaN,NaN,NaN,0.00,0,2022-01-03,35
446,HU,KNDJ23AU2N7176161,[N]Normal Port Stock,KMA,[S2000]Stock,01/03/2022,2022010300036,[SO1]SOUL (SK3),2022,[B2522]MY22 SOUL (SK3),15,"[EXTRM]Extreme Transportation, Inc.",[CA305B]KIA OF VACAVILLE,NaN,01/06/2022,[DS1872]AVIS RENT A CAR,Fleet Car,SAN DIEGO,CA,[WF][KMA] Fleet,[SWP]Snow White Pearl,[WK]Black,11/26/2021,2021-12-17,0017894844,2022-01-03,Y,[U]Unsold,NaN,Y,NaN,NaN,NaN,NaN

In [87]:
ant_vtype.sort_values('ANT Aging', ascending = False)

,Port,VIN,Status,Customer,VPS,Mani Aloc Date,Mani Aloc No.,Model Type,Model Year,Model,Acce Grup Cd,Carrier,Dealer,Dest,Req Port Rels Date,Ship To Dealer,Ship.Dealer Type,Ship to City,Ship to State,Region,Ext. Color,Int. Color,Production Date,Received,Allocation,Allocation Date,Allocated,Sold Status,Instruction,Drop Ship,VPC In Date,VPC Out Date,Release Date,Shipment Date,Shipment No.,Load No.,Load Date,Rail Load Position,Trans. Type,De-commit Date,PUP No.,PUP Date,PUP Open Flag,Activity Code,PIO Code,Install Type,HOLD,HOLD Code,HOLD Comment,R/O No.,R/O Date,Start Due Date,End Due Date,Work Order Print Date,Received Aged,Allocated Days Aged,comment,Location,Location Update,Location Update Date,Vessel,Vessel ETA,In Process Date,Arrival Date,Rail Car No.,XM ID,Inspector ID,Throw In ID,VEHICLE_TYPE,Light PIO Hrs,Heavy PIO Hrs,Start Date,ANT Aging
217,HU,KMHT36AH1NU011970,[N]Normal Port Stock,HMA,[S2000]Stock,09/08/2021,2021090800087,[VL1]VELOSTER (JS),2022,[FN353FT5]MY22 VELOSTER N (JS),1,[TBD]To Be Determined,[WE555]HMA WESTERN REG VEH SERVICES,NaN,NaN,[MRAA]RIVERSIDE AUTO AUCTION,Dealer Car,RIVERSIDE,CA,[WE][KMA/HMA] Western,[SAW]Lunar White,[NNB]Black,07/06/2021,2021-07-28,3524280,2021-09-08,Y,[U]Unsold,CLASS 2/USED *DO NOT SELL AS N,Y,08/03/2021 08:11:35,08/03/2021 09:33:38,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,NaN,"CN,CT,PC,RP,WL",NaN,No,NaN,No cmnt hold,NaN,NaN,NaN,NaN,01/21/2022 13:29:23,184,141,NaN,DZ-L2B7-041-002,jlamb,01/21/2022 15:20:43,COSCO TE,07/26/2021,09/09/2021,07/27/2021,NaN,NaN,Cecilia Perez,NaN,NaN,0.41,0,2021-09-08,152
198,HU,KM8JBDA26NU031593,[N]Normal Port Stock,HMA,[S2000]Stock,09/23/2021,2021111500029,[TP1]TUCSON PHEV (NX4),2022,[85432APT]MY22 TUCSON PHEV (NX4) - SEL,1,"[EXTRM]Extreme Transportation, Inc.",[CA172]LITHIA HYUNDAI OF FRESNO,NaN,NaN,[CA172]LITHIA HYUNDAI OF FRESNO,Dealer Car,FRESNO,CA,[WE][KMA/HMA] Western,[A5G]Amazon Gray,[MMH]Gray,08/13/2021,2021-09-07,3543426,2021-09-23,Y,[S]Sold,NaN,N,09/28/2021 12:51:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,97 NXMG01,"CF,CV,MG",Light PIO,No,NaN,No cmnt hold,NaN,NaN,NaN,NaN,09/28/2021 10:27:10,143,126,Repairs completed,1C-FENCE-DONE-001,aflores,01/25/2022 15:54:26,PRESTIGE,09/03/2021,09/27/2021,09/03/2021,NaN,NaN,NaN,NaN,NaN,0.32,0,2021-09-23,137
86,HU,KMUHBDSBXNU077278,[N]Normal Port Stock,HMA,[S2000]Stock,10/09/2021,2021113000024,[V81]GENESIS GV80 (JX1),2022,[V0422A45]MY22 GENESIS GV80 (JX1),4,"[EXTRM]Extreme Transportation, Inc.",[CA721]GENESIS OF SOUTH BAY,NaN,NaN,[CA721]GENESIS OF SOUTH BAY,Dealer Car,TORRANCE,CA,[WE][KMA/HMA] Western,[PH3]Vik Black,[UT2]Beige w/ Taupe,09/03/2021,2021-10-01,3559408,2021-10-09,Y,[U]Unsold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,97 IKPQ01 JX5CT1 JX5PU1 JXTH01 JXTHH1 MUGFK2,"CT,FK,PC,PQ,TH",Heavy PIO,No,NaN,No cmnt hold,NaN,NaN,NaN,NaN,01/04/2022 08:20:57,119,110,NaN,GM-BS-XX4-005,aflores,01/21/2022 09:58:20,MORNING,09/29/2021,10/13/2021,09/30/2021,NaN,NaN,NaN,NaN,NaN,0.24,0,2021-10-09,121
341,HU,KMUHCESC4NU075960,[N]Normal Port Stock,HMA,[S3300]Staging for Truck,10/09/2021,2021113000024,[V81]GENESIS GV80 (JX1),2022,[V0462A65]MY22 GENESIS GV80 (JX1),6,"[EXTRM]Extreme Transportation, Inc.",[CA727]GENESIS OF CORONA,NaN,NaN,[CA727]GENESIS OF CORONA,Dealer Car,CORONA,CA,[WE][KMA/HMA] Western,[HT7]Cardiff Green,[UT2]Beige w/ Taupe,08/27/2021,2021-10-01,3558865,2021-10-09,Y,[S]Sold,NaN,N,10/15/2021 14:26:32,10/15/2021 15:19:37,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,NaN,"CT,FK,MG,PC,PQ,RP,WL",NaN,No,NaN,No cmnt hold,NaN,NaN,NaN,NaN,01/27/2022 04:13:19,119,110,NaN,DZ-FEN-002-001,jlamb,01/22/2022 12:02:12,MORNING,09/29/2021,10/13/2021,09/30/2021,NaN,NaN,SANDRA ARIAS,NaN,NaN,0.69,0,2021-10-09,121
405,HU,KMUHCESC1NU078783,[N]Normal Port Stock,HMA,[S2000]Stock,10/09/2021,2021120800036,[V81]GENESIS GV80 (JX1),2022,[V0472A65]MY22 GENESIS GV80 (JX1),1,"[EXTRM]Extreme Transportation, Inc.",[CA717]GENESIS OF SANTA ROSA,NaN,NaN,[CA717]GENESIS OF SANTA ROSA,Dealer Car,SANTA ROSA,CA,[WE][KMA/H

In [89]:
with pd.ExcelWriter(r'C:\Users\gls06945\OneDrive - Hyundai Motor Group NA\Documents\0122 - PP Optimization (Personal)\PP Alg Output Files\HU ANT.xlsx') as writer1:
    ant_vtype.to_excel(writer1)